In [17]:
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.append("/Users/redstar/work/analysisHAL_miya/scripts/python_code-set/lib")
from common.io_data_bin          import input_bin_data_naxyzb, input_bin_data_nt
from common.statistics           import make_mean_err, make_JKsample
from fitting.fitfunc_type        import set_fitfunc_from_fname
from misc_QM.special_functions   import sph_harm_xyz, sph_harm_A1_xyz
from lattice.rotation_projection import rot_proj
from lattice.calc_laplacian      import lap, lap_4th_proc
from lattice.calc_potential      import calc_potential_t2, calc_potential_t2_num
from lattice.make_r_coordinates  import xyzdata_to_rdata, make_r_coord, make_rbin, divide_wave_L04, rbin_L04_separation
from lattice.misner_method       import Sph_harm_deco_misner
from sch_diffeq.solve_diffeq     import solve_sch_diff
from Tmatrix.convert_mat         import convert_TtoS
from Tmatrix.calc_phase_shift    import calc_phase_Sii, within_one
from sch_finiteV.solve_sch_finiteV import solve_sch_Fvol, solve_sch_Fvol_4th_prec
from multiprocessing             import Pool
# for fitting and phase shift calculation
from scipy.optimize import curve_fit
from sch_gauss_exp.solve_sch_GEM   import solve_sch_GEM

In [18]:
Lsize = 4
Tsize = 8

In [19]:
ls

2pt_k_4teave
2pt_k_4teave_before
2pt_pi_4teave
2pt_pi_4teave_before
NBS_conn_4teave
NBS_conn_4teave_before
NBS_sep_4teave
NBS_sep_4teave_before
consistency_check_pik_sepconn_multithread.ipynb


In [20]:
wave_bt_sep = np.fromfile("NBS_sep_4teave_before",'<c16').reshape(Tsize, Lsize, Lsize, Lsize)

In [21]:
wave_at_sep = np.fromfile("NBS_sep_4teave",'<c16').reshape(Tsize, Lsize, Lsize, Lsize)

In [22]:
wave_bt_sep - wave_at_sep

array([[[[-6.23003871e-11+1.13159485e-15j,
          -6.23003871e-11+8.83516303e-16j,
          -6.04813977e-11+7.79662825e-16j,
          -6.23003871e-11+9.39027454e-16j],
         [-6.09361450e-11+1.17839432e-15j,
          -6.00266503e-11+9.32353345e-16j,
          -5.86624083e-11+7.34088267e-16j,
          -5.95719030e-11+1.04164092e-15j],
         [-5.95719030e-11+1.31894499e-15j,
          -5.82076609e-11+1.08994840e-15j,
          -5.72981662e-11+7.83132272e-16j,
          -5.82076609e-11+1.03443725e-15j],
         [-6.09361450e-11+1.17839432e-15j,
          -5.95719030e-11+1.04164092e-15j,
          -5.86624083e-11+7.34088267e-16j,
          -6.00266503e-11+9.32353345e-16j]],

        [[-6.09361450e-11+1.06949359e-15j,
          -6.04813977e-11+1.09359338e-15j,
          -5.95719030e-11+7.75241121e-16j,
          -5.91171556e-11+9.82571075e-16j],
         [-6.00266503e-11+1.17204142e-15j,
          -5.95719030e-11+9.91957818e-16j,
          -5.86624083e-11+6.66586368e-16j,
    

In [23]:
(wave_bt_sep.real - wave_at_sep.real) / wave_bt_sep.real

array([[[[-2.98419723e-14, -2.98186877e-14, -2.89282863e-14,
          -2.98186877e-14],
         [-2.92177718e-14, -2.87237761e-14, -2.80481561e-14,
          -2.85196009e-14],
         [-2.85826716e-14, -2.78634410e-14, -2.73841765e-14,
          -2.78634410e-14],
         [-2.92177718e-14, -2.85196009e-14, -2.80481561e-14,
          -2.87237761e-14]],

        [[-2.92475897e-14, -2.89930380e-14, -2.85202335e-14,
          -2.83400437e-14],
         [-2.88131304e-14, -2.85559492e-14, -2.80826279e-14,
          -2.83344830e-14],
         [-2.83670359e-14, -2.76614481e-14, -2.76390686e-14,
          -2.78949267e-14],
         [-2.85811626e-14, -2.81013953e-14, -2.78595093e-14,
          -2.81191146e-14]],

        [[-2.81998393e-14, -2.81574130e-14, -2.81159392e-14,
          -2.81574130e-14],
         [-2.81769029e-14, -2.79370877e-14, -2.79040185e-14,
          -2.81349192e-14],
         [-2.79340479e-14, -2.79210134e-14, -2.76826174e-14,
          -2.79210134e-14],
         [-2.8176

In [24]:
(wave_bt_sep.imag - wave_at_sep.imag) / wave_bt_sep.imag

/Users/redstar/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[[[ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf]],

        [[ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf]],

        [[ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf]],

        [[ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf],
         [ inf,  inf,  inf,  inf]]],


       [[[-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf]],

        [[-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf]],

        [[-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf]],

    

In [25]:
wave_bt_conn = np.fromfile("NBS_conn_4teave_before",'<c16').reshape(Tsize, Lsize, Lsize, Lsize)

In [26]:
wave_at_conn = np.fromfile("NBS_conn_4teave",'<c16').reshape(Tsize, Lsize, Lsize, Lsize)

In [27]:
wave_bt_conn

array([[[[-1.81220756e+02+0.00000000e+00j,
          -1.11720854e+02+4.60742555e-15j,
          -6.98123512e+01+0.00000000e+00j,
          -1.11720854e+02-4.60742555e-15j],
         [-1.12683510e+02-1.05471187e-15j,
          -8.08425327e+01+3.22658567e-15j,
          -5.49922127e+01-2.04003481e-15j,
          -8.17162412e+01-7.52176099e-15j],
         [-7.18393326e+01+0.00000000e+00j,
          -5.60313940e+01+5.02375919e-15j,
          -4.21042673e+01+0.00000000e+00j,
          -5.60313940e+01-5.02375919e-15j],
         [-1.12683510e+02+1.05471187e-15j,
          -8.17162412e+01+7.52176099e-15j,
          -5.49922127e+01+2.04003481e-15j,
          -8.08425327e+01-3.22658567e-15j]],

        [[-1.10224436e+02-1.77635684e-15j,
          -7.91295682e+01+1.12410081e-15j,
          -5.40472912e+01-4.44089210e-15j,
          -7.99622577e+01-7.79931675e-15j],
         [-8.05155769e+01-3.08086889e-15j,
          -6.02572195e+01+1.85962357e-15j,
          -4.39435574e+01-5.63438185e-15j,
    

In [28]:
wave_bt_conn - wave_at_conn

array([[[[7.61701813e-12+8.85091529e-17j,
          6.60804744e-12+1.81799020e-15j,
          6.03961325e-12+1.22915356e-16j,
          6.60804744e-12-1.33226763e-15j],
         [6.13908924e-12+2.60902411e-15j,
          5.96855898e-12+5.73152636e-15j,
          5.24380539e-12-9.29811783e-16j,
          5.32907052e-12-2.42167397e-15j],
         [5.41433565e-12+2.41731235e-16j,
          4.88853402e-12+4.52415883e-15j,
          4.39825953e-12+3.18319335e-16j,
          4.88853402e-12-3.91353616e-15j],
         [6.13908924e-12-2.16493490e-15j,
          5.32907052e-12+2.87964097e-15j,
          5.24380539e-12+1.20736754e-15j,
          5.96855898e-12-5.37070388e-15j]],

        [[6.01119154e-12-2.61075883e-15j,
          6.08224582e-12-1.17961196e-15j,
          5.44986278e-12-3.19883009e-15j,
          5.31485966e-12-5.32907052e-15j],
         [5.44275736e-12+6.38378239e-16j,
          5.57065505e-12+3.96904731e-15j,
          4.76774176e-12-1.80411242e-15j,
          4.52615723e-12-4.

In [29]:
(wave_bt_conn.real - wave_at_conn.real) / wave_bt_conn.real

array([[[[-4.20317093e-14, -5.91478420e-14, -8.65121020e-14,
          -5.91478420e-14],
         [-5.44808129e-14, -7.38294408e-14, -9.53554172e-14,
          -6.52143373e-14],
         [-7.53672877e-14, -8.72463395e-14, -1.04461135e-13,
          -8.72463395e-14],
         [-5.44808129e-14, -6.52143373e-14, -9.53554172e-14,
          -7.38294408e-14]],

        [[-5.45359249e-14, -7.68643878e-14, -1.00835077e-13,
          -6.64671036e-14],
         [-6.75988122e-14, -9.24479273e-14, -1.08496946e-13,
          -7.30241815e-14],
         [-9.48169235e-14, -1.15220023e-13, -1.26212579e-13,
          -1.02133153e-13],
         [-7.15628340e-14, -8.75841519e-14, -1.16910111e-13,
          -9.02242661e-14]],

        [[-7.80050957e-14, -9.63048589e-14, -1.24118559e-13,
          -9.63048589e-14],
         [-1.00146683e-13, -1.26863908e-13, -1.40421808e-13,
          -1.07546685e-13],
         [-1.26963039e-13, -1.36431424e-13, -1.51060538e-13,
          -1.36431424e-13],
         [-1.0014

In [30]:
(wave_bt_conn.imag - wave_at_conn.imag) / wave_bt_conn.imag

/Users/redstar/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[[[            inf,  3.94578313e-01,             inf,
           2.89156627e-01],
         [-2.47368421e+00,  1.77634409e+00,  4.55782313e-01,
           3.21955720e-01],
         [            inf,  9.00552486e-01,             inf,
           7.79005525e-01],
         [-2.05263158e+00,  3.82841328e-01,  5.91836735e-01,
           1.66451613e+00]],

        [[ 1.46972656e+00, -1.04938272e+00,  7.20312500e-01,
           6.83274021e-01],
         [-2.07207207e-01,  2.13432836e+00,  3.20197044e-01,
           4.03914591e-01],
         [ 1.61764706e+00,  7.01492537e-01,  2.64802632e-01,
           7.20333547e-01],
         [ 3.43750000e+00, -2.73626374e+00,  2.91866029e-01,
           1.05992509e+00]],

        [[            inf, -5.39215686e-02,             inf,
          -1.86274510e-01],
         [-2.15833333e+01,  1.15887850e+00, -2.65789474e+00,
          -2.46913580e-02],
         [            inf,  7.27124183e-01,             inf,
           6.96078431e-01],
         [-2.0416